## Совместная фильтрация элемент-элемент
1. Оффлайн часть
1.1. Получаем оценки всех пользователей
1.2. Нормализуем их (столбец mean)
1.3. Заменяем id пользователей и фильмов на категории (например фильм с id 100111 получает категорию 0, фильм c id 100112 получает категорию 1 и так далее)
1.4. Объединяем колонку mean, колонку категорий пользователей и колонку категорий фильм в разряженную матрицу coo
1.5. Создаём матрицу перекрытия, чтобы исключить фильмы у которых мало оценок. Матрица перекрытия показывает сколько пользователей одновременно оценили фильм x и фильм y. 
1.6. Получаем матрицу сходства между фильмами, для вычисления сходство используется косиносовое сходство. cor = cosine_similarity
1.7. В матрице сходства обнуляем ячейки у которых низкое сходство и укоторых маленькое перекрытие
1.8. Из полученной матрице сходств формируем таблицу сходств фильмов (similarity_table), в которой категории фильмов заменены на их настоящие id 
2. Онлайн часть
2.1. Получаем оценки текущего пользователя
2.2. Сортируем оценки по самым высоким
2.3. Вычисляем среднюю оценку пользователя (current_user_mean)
2.4. Из таблицы сходств получаем только те фильмы которые оценены текущим пользователем
2.5. Из таблицы сходств исключаем фильмы которые оценены текущим пользователем и находятся в колонки target_id. То есть мы не будем рекомендовать пользователю фильмы, которые он уже посмотрел.
2.6. Вычисляем рекомендации для пользователя, и получаем таблицу вида: target_id; prediction;sim_movies;
где target_id - id фильма рекомендуемого пользователю; prediction - ожидаемая оценка данному фильму; sim_movies- на основание каких фильмов оцененных пользователем ранее выдана данная рекомендация.

In [50]:
import requests
import time
import pandas as pd
import surprise as surprise
import numpy as np
from decimal import Decimal
from scipy.sparse import coo_matrix
from scipy import sparse
# косинусовое сходство
from sklearn.metrics.pairwise import cosine_similarity

In [51]:
users_ratings = pd.DataFrame([
    [0,10,5.0],
    [0,11,3.0],
    [0,13,2.0],
    [0,14,2.0],
    [0,15,2.0],
    
    [1,10,4.0],
    [1,11,3.0],
    [1,12,4.0],
    [1,14,3.0],
    [1,15,3.0],
    
     [2,10,5.0],
    [2,11,2.0],
    [2,12,5.0],
     [2,13,2.0],
    [2,14,1.0],
    [2,15,1.0],
    
      [3,10,3.0],
    [3,11,5.0],
    [3,12,3.0],
    [3,14,1.0],
    [3,15,1.0],
    
      [4,10,3.0],
    [4,11,3.0],
    [4,12,3.0],
     [4,13,2.0],
    [4,14,4.0],
    [4,15,5.0],
    
      [5,10,2.0],
    [5,11,3.0],
    [5,12,2.0],
     [5,13,3.0],
    [5,14,5.0],
    [5,15,5.0],
],columns=['user_id','movie_id','rating'])

In [52]:
# Нормализуем оценки вычтя из оценки среднюю оценку пользователя
def normalize(x):
    x = x.astype(float)
    x_sum = x.sum()
    x_num = x.astype(bool).sum()
    x_mean = x_sum / x_num

    if x_num == 1 or x.std() == 0:
        return 0.0
    return (x - x_mean) / (x.max() - x.min())

In [53]:
mean_col = users_ratings.groupby(['user_id'])['rating'].mean()
max_col = users_ratings.groupby(['user_id'])['rating'].max()
min_col = users_ratings.groupby(['user_id'])['rating'].min()
print(mean_col)
# users_ratings['mean'] = mean_col
# users_ratings.reset_index()
# print(users_ratings)

for index, row in users_ratings.iterrows():
    users_ratings.at[index,'mean']=(row['rating']-mean_col.at[row['user_id']])    /(max_col.at[row['user_id']]-min_col.at[row['user_id']])
    
users_ratings

user_id
0    2.800000
1    3.400000
2    2.666667
3    2.600000
4    3.333333
5    3.333333
Name: rating, dtype: float64


,user_id,movie_id,rating,mean
0,0,10,5.0,0.733333
1,0,11,3.0,0.066667
2,0,13,2.0,-0.266667
3,0,14,2.0,-0.266667
4,0,15,2.0,-0.266667
5,1,10,4.0,0.600000
6,1,11,3.0,-0.400000
7,1,12,4.0,0.600000
8,1,14,3.0,-0.400000
9,1,15,3.0,-0.400000


In [54]:
# from pandas.api.types import CategoricalDtype
# cat_dtype = CategoricalDtype(categories=[0,1,2,3,4,5], ordered=True)

users_values = users_ratings['user_id'].sort_values().unique()
movies_values = users_ratings['movie_id'].sort_values().unique()
print(movies_values)
user_categories = dict(enumerate(users_ratings['user_id'].sort_values().unique()))
movie_categories = dict(enumerate(users_ratings['movie_id'].sort_values().unique()))
print(movie_categories)
for index, row in users_ratings.iterrows():
    users_ratings.at[index,'movie_id'] = np.where(movies_values == row['movie_id'])[0]
    users_ratings.at[index,'user_id'] = np.where(users_values == row['user_id'])[0]

# users_ratings['user_id'] = users_ratings['user_id'].astype(cat_dtype)
# users_ratings['movie_id'] = users_ratings['movie_id'].astype(cat_dtype)
print(users_ratings)

[10 11 12 13 14 15]
{0: 10, 1: 11, 2: 12, 3: 13, 4: 14, 5: 15}
    user_id  movie_id  rating      mean
0         0         0     5.0  0.733333
1         0         1     3.0  0.066667
2         0         3     2.0 -0.266667
3         0         4     2.0 -0.266667
4         0         5     2.0 -0.266667
5         1         0     4.0  0.600000
6         1         1     3.0 -0.400000
7         1         2     4.0  0.600000
8         1         4     3.0 -0.400000
9         1         5     3.0 -0.400000
10        2         0     5.0  0.583333
11        2         1     2.0 -0.166667
12        2         2     5.0  0.583333
13        2         3     2.0 -0.166667
14        2         4     1.0 -0.416667
15        2         5     1.0 -0.416667
16        3         0     3.0  0.100000
17        3         1     5.0  0.600000
18        3         2     3.0  0.100000
19        3         4     1.0 -0.400000
20        3         5     1.0 -0.400000
21        4         0     3.0 -0.111111
22        4      

In [55]:
coo = coo_matrix((users_ratings['mean'],
                          (users_ratings['movie_id'],users_ratings['user_id'])
                 ))

In [56]:
print(coo)

  (0, 0)	0.7333333333333334
  (1, 0)	0.06666666666666672
  (3, 0)	-0.2666666666666666
  (4, 0)	-0.2666666666666666
  (5, 0)	-0.2666666666666666
  (0, 1)	0.6000000000000001
  (1, 1)	-0.3999999999999999
  (2, 1)	0.6000000000000001
  (4, 1)	-0.3999999999999999
  (5, 1)	-0.3999999999999999
  (0, 2)	0.5833333333333334
  (1, 2)	-0.16666666666666663
  (2, 2)	0.5833333333333334
  (3, 2)	-0.16666666666666663
  (4, 2)	-0.41666666666666663
  (5, 2)	-0.41666666666666663
  (0, 3)	0.09999999999999998
  (1, 3)	0.6
  (2, 3)	0.09999999999999998
  (4, 3)	-0.4
  (5, 3)	-0.4
  (0, 4)	-0.11111111111111116
  (1, 4)	-0.11111111111111116
  (2, 4)	-0.11111111111111116
  (3, 4)	-0.4444444444444445
  (4, 4)	0.22222222222222218
  (5, 4)	0.5555555555555555
  (0, 5)	-0.4444444444444445
  (1, 5)	-0.11111111111111116
  (2, 5)	-0.4444444444444445
  (3, 5)	-0.11111111111111116
  (4, 5)	0.5555555555555555
  (5, 5)	0.5555555555555555


In [57]:
print(coo.transpose().astype(bool).astype(int))

  (0, 0)	1
  (1, 0)	1
  (2, 0)	1
  (3, 0)	1
  (4, 0)	1
  (5, 0)	1
  (0, 1)	1
  (1, 1)	1
  (2, 1)	1
  (3, 1)	1
  (4, 1)	1
  (5, 1)	1
  (1, 2)	1
  (2, 2)	1
  (3, 2)	1
  (4, 2)	1
  (5, 2)	1
  (0, 3)	1
  (2, 3)	1
  (4, 3)	1
  (5, 3)	1
  (0, 4)	1
  (1, 4)	1
  (2, 4)	1
  (3, 4)	1
  (4, 4)	1
  (5, 4)	1
  (0, 5)	1
  (1, 5)	1
  (2, 5)	1
  (3, 5)	1
  (4, 5)	1
  (5, 5)	1


In [58]:
overlap_matrix = coo.astype(bool).astype(int).dot(coo.transpose().astype(bool).astype(int))
min_overlap=4
number_of_overlaps = (overlap_matrix > min_overlap).count_nonzero()

## Матрица перекрытия
Матрица перекрытия показывает сколько пользователей одновременно оценили фильм x и фильм y. 
Например цифра 3 показывает сколько людей одновременно оценили Храброе сердце и Эйс-Вентура.
Матрица смимметричная.
min_overlap=4
Мы можем не учитывать элементы, у которых мало оценок (например <= 4), так как это можно привести к неправильному значению сходства между этими фильмами.
number_of_overlaps - показывает сколько элементов осталось после примененения матрицы перекрытия

In [59]:
overlap_matrix.toarray()

array([[6, 6, 5, 4, 6, 6],
       [6, 6, 5, 4, 6, 6],
       [5, 5, 5, 3, 5, 5],
       [4, 4, 3, 4, 4, 4],
       [6, 6, 5, 4, 6, 6],
       [6, 6, 5, 4, 6, 6]], dtype=int32)

In [60]:
number_of_overlaps

25

In [61]:
# по столбцам пользователи по строкам фильмы
# нули отсутсвующие фильмы у пользователей
coo.toarray()

array([[ 0.73333333,  0.6       ,  0.58333333,  0.1       , -0.11111111,
        -0.44444444],
       [ 0.06666667, -0.4       , -0.16666667,  0.6       , -0.11111111,
        -0.11111111],
       [ 0.        ,  0.6       ,  0.58333333,  0.1       , -0.11111111,
        -0.44444444],
       [-0.26666667,  0.        , -0.16666667,  0.        , -0.44444444,
        -0.11111111],
       [-0.26666667, -0.4       , -0.41666667, -0.4       ,  0.22222222,
         0.55555556],
       [-0.26666667, -0.4       , -0.41666667, -0.4       ,  0.55555556,
         0.55555556]])

In [62]:
M=coo.toarray()


def calculate_cosine_distance_cor(M):
    Res = np.empty(M.shape)
    def my_cosine_distance(i,j):
        return np.sum(M[i,:]*M[j,:])/(np.sqrt(np.sum(np.square(M[i,:])))*np.sqrt(np.sum(np.square(M[j,:]))))
    for i in range(M.shape[0]):
        for j in range(i,M.shape[0]):
            calculated = my_cosine_distance(i,j);
            if (i==j):
                Res[i,j]=calculated
            else:
                Res[i,j]=Res[j,i]=calculated
    return sparse.csr_matrix(Res)

cor1 = calculate_cosine_distance_cor(M)
print(type(cor1))
min_sim=0.2
# Отбрасываем элементы с низким сходством (-1 фильмы совершенно непохожи, 1 фильмы одинаковы)
cor1 = cor1.multiply(cor1 > min_sim)
print(cor1.toarray())
cor1 = cor1.multiply(overlap_matrix > min_overlap)

<class 'scipy.sparse.csr.csr_matrix'>
[[1.         0.         0.79444107 0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.        ]
 [0.79444107 0.         1.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.95447817]
 [0.         0.         0.         0.         0.95447817 1.        ]]


In [63]:
# Вычисляем сходство
cor = cosine_similarity(coo, dense_output=False)
print(type(cor))
print(cor.toarray())
min_sim=0.2
# Отбрасываем элементы с низким сходством (-1 фильмы совершенно непохожи, 1 фильмы одинаковы)
cor = cor.multiply(cor > min_sim)
print(cor.toarray())
cor = cor.multiply(overlap_matrix > min_overlap)
print(cor.toarray())

<class 'scipy.sparse.csr.csr_matrix'>
[[ 1.         -0.18166189  0.79444107 -0.28916544 -0.85372896 -0.78252628]
 [-0.18166189  1.         -0.29576659  0.16995009 -0.1572779  -0.18381386]
 [ 0.79444107 -0.29576659  1.          0.00289521 -0.86240246 -0.79748992]
 [-0.28916544  0.16995009  0.00289521  1.         -0.03735355 -0.27823286]
 [-0.85372896 -0.1572779  -0.86240246 -0.03735355  1.          0.95447817]
 [-0.78252628 -0.18381386 -0.79748992 -0.27823286  0.95447817  1.        ]]
[[1.         0.         0.79444107 0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.        ]
 [0.79444107 0.         1.         0.         0.         0.        ]
 [0.         0.         0.         1.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.95447817]
 [0.         0.         0.         0.         0.95447817 1.        ]]
[[1.         0.         0.79444107 0.         0.         0.        ]
 [0.         1.         0. 

In [64]:
cor.toarray()

array([[1.        , 0.        , 0.79444107, 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.79444107, 0.        , 1.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 1.        ,
        0.95447817],
       [0.        , 0.        , 0.        , 0.        , 0.95447817,
        1.        ]])

In [65]:
print(dict(enumerate(users_ratings['movie_id'].unique())))

{0: 0, 1: 1, 2: 3, 3: 4, 4: 5, 5: 2}


In [69]:
movie_categories

{0: 10, 1: 11, 2: 12, 3: 13, 4: 14, 5: 15}

In [66]:
def save_sims(cor,movies_categories):
    coo = coo_matrix(cor)
    print(coo)
    csr = coo.tocsr()
    
    no_saved = 0
    min_sim=0.1
    xs, ys = coo.nonzero()

    columns = ['source_id', 'target_id', 'similarity']
    M = pd.DataFrame([],columns=columns)
#     movies_ids = df['movie_id'].sort_values().unique()
          
#     print(df['movie_id'].astype('category'))
    for x, y in zip(xs, ys):
    #     x это первый элемент кортежа, y второй; x,y это id строк описаний между которыми вычисляется сходство
        print(x,y)
    # нет смысл сравнить строку описания с самой собой это вернёт 1 или близко к этому
        if x == y:
            continue
        sim = float(csr[x, y])
        if sim < min_sim:
            continue
        M.loc[M.shape[0]] = [int(movies_categories[x]),int(movies_categories[y]),sim]
       
        no_saved += 1
   
    M['source_id'] =  M['source_id'].astype(int)
    M['target_id'] =  M['target_id'].astype(int)
    return M

In [67]:
similarity_table = save_sims(cor,movie_categories)
print(similarity_table)

  (0, 0)	0.9999999999999999
  (0, 2)	0.7944410670491047
  (1, 1)	0.9999999999999996
  (2, 0)	0.7944410670491047
  (2, 2)	1.0
  (4, 4)	1.0
  (4, 5)	0.9544781707855685
  (5, 4)	0.9544781707855685
  (5, 5)	0.9999999999999998
0 0
0 2
1 1
2 0
2 2
4 4
4 5
5 4
5 5
   source_id  target_id  similarity
0         10         12    0.794441
1         12         10    0.794441
2         14         15    0.954478
3         15         14    0.954478


In [24]:
#онлайн часть

In [29]:
# получаем оценки текущего пользователя
columns = ['user_id', 'movie_id', 'rating']
current_user_ratings = pd.DataFrame([
    ['100',10,4],
     ['100',11,3],
     ['100',14,5],
],columns=columns)

current_user_mean = current_user_ratings['rating'].mean()

# сортируем оценки по самым высоким
current_user_ratings.sort_values(['rating'], ascending=[0],inplace=True)
current_user_ratings.reset_index(drop=True, inplace=True)
print(current_user_ratings)

# получаем только те фильмы которые оцнены текущим пользователем и находятся в таблице сходств
in_source=similarity_table['source_id'].isin(current_user_ratings['movie_id']) 
# Исключаем из таблице сходств уже оцененные фильмы пользователи (не нужно рекомендовать, то, что уже было просмотренно)
not_in_target=~similarity_table['target_id'].isin(current_user_ratings['movie_id'])

# Получаем таблиц сходств для текущего пользователя
top_by_sims = similarity_table.loc[in_source &  not_in_target].sort_values(['similarity'], ascending=[0])[:30]
top_by_sims.reset_index(drop=True, inplace=True)
print(top_by_sims)

columns = ['target_id', 'prediction', 'sim_movies']
recs = pd.DataFrame([],columns=columns)
targets = top_by_sims['target_id']

for target_row in targets:
    pre = 0
    sim_sum = 0
#     print(target_row)
    rated_items = top_by_sims[top_by_sims['target_id']==target_row]
#     print(rated_items)
    if len(rated_items) > 0:
        for index, sim_item in rated_items.iterrows():
#             получаем оценку пользователя для фильма
            user_rating = current_user_ratings[current_user_ratings['movie_id']==sim_item['source_id']].iloc[0]['rating']
#         вычитаем из оценки среднюю для пользователя
            r = user_rating- current_user_mean
#     умножаем схожесть на оценку
            pre += sim_item.similarity * r
            sim_sum += sim_item.similarity
        if sim_sum > 0:
#               print(rated_items['source_id'].values)
# формируем прогноз
            recs.loc[recs.shape[0]] = [target_row,current_user_mean + pre/sim_sum,rated_items['source_id'].values]
#             recs[target] = { 'prediction': Decimal(user_mean) + pre/sim_sum,'sim_items': [r.source for r in rated_items]}
# return sorted(recs.items(),key=lambda item: -float(item[1]['prediction']))[:num]
recs.sort_values(['prediction'], ascending=[0],inplace=True)
recs.reset_index(drop=True, inplace=True)
print(recs)


  user_id  movie_id  rating
0     100        14       5
1     100        10       4
2     100        11       3
   source_id  target_id  similarity
0         14         15    0.954478
1         10         12    0.794441
  target_id  prediction sim_movies
0        15         5.0       [14]
1        12         4.0       [10]
